# Project 3
## B-IT Pattern Recognition



Presented on 4-Jan-2016 by:

- Abdullah Abdullah

- Can Güney Aksakallı

- Kang Cifong

- Umut Hatipoğlu

***

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import pattrex.plotting_mpl as plt_rex
# import pattrex.dimred as dim_rex

## Task 3.1
### Fun with k-means clustering
***

## Task 3.2
### Spectral Clustering
***

## Task 3.3
### Dimensionality Reduction
***

## Task 3.4
### Exploring Numerical Instabilities
***

## References

- 

***
## Questions?